### Eventloop (Lösung)

In [ ]:
import threading
from ipywidgets import Output, Button
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)
button_stop = Button(description='Stop')
button_new_game = Button(description='New Game')

state = {'pos': (50, 50)}
key_move = {'ArrowUp': (0, -5),
            'ArrowDown': (0, 5),
            'ArrowLeft': (-5, 0),
            'ArrowRight': (5, 0),
            }

stop_event = threading.Event()
event_queue = []


@out.capture()
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        event_queue.insert(0, key)


def move(dx, dy):
    x0, y0 = state['pos']
    x, y = (x0+dx, y0+dy)

    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)

    if x < 0 or y < 0 or x > 100 or y > 100:
        canvas.fill_rect(0, 0, canvas.width, canvas.height)
        stop_event.set()


@out.capture()
def new_game():
    canvas.clear()
    stop_event.clear()
    event_queue.clear()
    event_queue.insert(0, 'Right')
    event_loop()
    canvas.focus()


def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(0.3, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()


canvas.on_key_down(on_key_down)
button_stop.on_click(lambda bt: stop_event.set())
button_new_game.on_click(lambda bt: new_game())

display(canvas, button_stop, button_new_game, out)
canvas.focus()  # Canvas-Widget hoert auf Events, ohne das es zuerst angeklickt werden muss